# Governance with Horizon (PATELCO Credit Union Demo)

This notebook sets up production-grade governance for PII that automatically applies across Snowflake and Cortex (Analyst, Agents, Search, Intelligence). It is intentionally simple: apply controls, then show them working for two roles.

Sections:
1. Roles and Access Control
2. Tag-Based Classification (manual)
3. Column-level Security (masking)
4. Row Level Security (RLS)
5. Data Quality (standard SQL)
6. Trust Center (how to review)



In [ ]:
/*
***************************************************************************************************       
Asset:        Zero to Snowflake - Governance with Horizon (PATELCO Demo)
Version:      v1     
Copyright(c): 2025 Snowflake Inc. All rights reserved.
****************************************************************************************************

Governance with Horizon - PATELCO Credit Union Demo
1. Introduction to Roles and Access Control
2. Tag-Based Classification with Manual Tagging
3. Column-level Security with Masking Policies
4. Row Level Security with Row Access Policies
5. Data Quality Monitoring with Standard SQL Checks
6. Account Security Monitoring with the Trust Center

This notebook sets up production-grade governance for PII that automatically applies 
across all Snowflake tools, including Cortex (Analyst, Agents, Search, Intelligence).

****************************************************************************************************/

-- Set the session query tag
ALTER SESSION SET query_tag = '{"origin":"sf_demo","name":"governance_horizon_patelco","version":{"major":1,"minor":0}}';

-- Set our Worksheet context
USE ROLE accountadmin;
USE DATABASE PATELCO_DEMO;
USE SCHEMA CREDIT_UNION_DATA;
USE WAREHOUSE COMPUTE_WH;


In [ ]:
/*  1. Introduction to Roles and Access Control  */

-- Create custom roles for credit union governance (ACCOUNTADMIN)
USE ROLE accountadmin;
USE DATABASE PATELCO_DEMO;
USE SCHEMA CREDIT_UNION_DATA;
USE WAREHOUSE COMPUTE_WH;

CREATE OR REPLACE ROLE compliance_officer COMMENT = 'Full access for regulatory oversight';
CREATE OR REPLACE ROLE data_analyst COMMENT = 'Restricted analytics role with PII protection';


In [ ]:
-- Environment prep (ACCOUNTADMIN): ensure objects exist
USE ROLE accountadmin;

-- Database and schema (idempotent)
CREATE DATABASE IF NOT EXISTS PATELCO_DEMO;
CREATE SCHEMA   IF NOT EXISTS PATELCO_DEMO.CREDIT_UNION_DATA;

-- Warehouse (idempotent)
CREATE WAREHOUSE IF NOT EXISTS COMPUTE_WH
  WAREHOUSE_SIZE = 'SMALL'
  AUTO_SUSPEND   = 60
  AUTO_RESUME    = TRUE;
ALTER WAREHOUSE COMPUTE_WH RESUME IF SUSPENDED;


In [ ]:
-- Grants (ACCOUNTADMIN to avoid org-level restrictions)
USE ROLE accountadmin;
USE DATABASE PATELCO_DEMO;
USE SCHEMA CREDIT_UNION_DATA;
ALTER WAREHOUSE COMPUTE_WH RESUME IF SUSPENDED;
USE WAREHOUSE COMPUTE_WH;

GRANT USAGE ON WAREHOUSE COMPUTE_WH TO ROLE compliance_officer;
GRANT USAGE ON WAREHOUSE COMPUTE_WH TO ROLE data_analyst;

GRANT USAGE ON DATABASE PATELCO_DEMO                 TO ROLE compliance_officer;
GRANT USAGE ON DATABASE PATELCO_DEMO                 TO ROLE data_analyst;
GRANT USAGE ON SCHEMA PATELCO_DEMO.CREDIT_UNION_DATA TO ROLE compliance_officer;
GRANT USAGE ON SCHEMA PATELCO_DEMO.CREDIT_UNION_DATA TO ROLE data_analyst;

GRANT SELECT ON ALL TABLES IN SCHEMA PATELCO_DEMO.CREDIT_UNION_DATA TO ROLE compliance_officer;
GRANT SELECT ON ALL TABLES IN SCHEMA PATELCO_DEMO.CREDIT_UNION_DATA TO ROLE data_analyst;


In [ ]:
-- Assign roles to current user (SECURITYADMIN)
USE ROLE securityadmin;
-- No warehouse needed for role grants

SET my_user = CURRENT_USER();
GRANT ROLE compliance_officer TO USER IDENTIFIER($my_user);
GRANT ROLE data_analyst       TO USER IDENTIFIER($my_user);

-- Quick test (COMPLIANCE_OFFICER)
USE ROLE compliance_officer;
USE DATABASE PATELCO_DEMO; USE SCHEMA CREDIT_UNION_DATA; USE WAREHOUSE COMPUTE_WH;
SELECT TOP 5 member_id, first_name, last_name, state FROM members;


In [ ]:
/*  2. Tag-Based Classification with Manual Tagging  */

USE ROLE accountadmin;

-- Create governance schema and PII tag
CREATE SCHEMA IF NOT EXISTS governance;
CREATE OR REPLACE TAG governance.pii COMMENT = 'Sensitive PII data requiring protection';

-- Grant tag application privileges
GRANT APPLY TAG ON ACCOUNT TO ROLE compliance_officer;
GRANT ALL ON SCHEMA governance TO ROLE compliance_officer;

-- Switch to compliance officer to apply tags
USE ROLE compliance_officer;

ALTER TABLE members MODIFY COLUMN first_name SET TAG governance.pii = 'sensitive';
ALTER TABLE members MODIFY COLUMN last_name  SET TAG governance.pii = 'sensitive';
ALTER TABLE members MODIFY COLUMN email      SET TAG governance.pii = 'sensitive';
ALTER TABLE members MODIFY COLUMN phone      SET TAG governance.pii = 'sensitive';


In [ ]:
/*  3. Column-level Security with Masking Policies  */

USE ROLE accountadmin;

-- Remove any existing masking policies (idempotent)
ALTER TABLE members ALTER COLUMN first_name UNSET MASKING POLICY;
ALTER TABLE members ALTER COLUMN last_name UNSET MASKING POLICY;
ALTER TABLE members ALTER COLUMN email UNSET MASKING POLICY;
ALTER TABLE members ALTER COLUMN phone UNSET MASKING POLICY;
ALTER TABLE members ALTER COLUMN annual_income UNSET MASKING POLICY;
ALTER TABLE members ALTER COLUMN credit_score UNSET MASKING POLICY;

-- Create masking policies
CREATE OR REPLACE MASKING POLICY governance.mask_string_pii AS (original_value STRING)
RETURNS STRING ->
  CASE WHEN CURRENT_ROLE() IN ('ACCOUNTADMIN', 'COMPLIANCE_OFFICER') THEN original_value ELSE '****MASKED****' END;

CREATE OR REPLACE MASKING POLICY governance.mask_number_pii AS (original_value NUMBER)
RETURNS NUMBER ->
  CASE WHEN CURRENT_ROLE() IN ('ACCOUNTADMIN', 'COMPLIANCE_OFFICER') THEN original_value ELSE NULL END;

-- Apply masking policies
ALTER TABLE members ALTER COLUMN first_name    SET MASKING POLICY governance.mask_string_pii;
ALTER TABLE members ALTER COLUMN last_name     SET MASKING POLICY governance.mask_string_pii;
ALTER TABLE members ALTER COLUMN email         SET MASKING POLICY governance.mask_string_pii;
ALTER TABLE members ALTER COLUMN phone         SET MASKING POLICY governance.mask_string_pii;
ALTER TABLE members ALTER COLUMN annual_income SET MASKING POLICY governance.mask_number_pii;
ALTER TABLE members ALTER COLUMN credit_score  SET MASKING POLICY governance.mask_number_pii;

-- Action demo: masked vs unmasked
USE ROLE COMPLIANCE_OFFICER; USE WAREHOUSE COMPUTE_WH;
SELECT member_id, first_name, last_name, email, annual_income
FROM members ORDER BY member_id LIMIT 5;  -- unmasked

USE ROLE DATA_ANALYST; USE WAREHOUSE COMPUTE_WH;
SELECT member_id, first_name, last_name, email, annual_income
FROM members ORDER BY member_id LIMIT 5;  -- strings masked, numbers NULL


In [ ]:
/*  4. Row Level Security with Row Access Policies  */

USE ROLE accountadmin;
USE DATABASE PATELCO_DEMO;
USE SCHEMA credit_union_data;
USE WAREHOUSE COMPUTE_WH;

-- Remove existing policies
ALTER TABLE members DROP ALL ROW ACCESS POLICIES;
DROP ROW ACCESS POLICY IF EXISTS governance.city_access_policy;

-- Create RLS policy with role and city check
CREATE OR REPLACE ROW ACCESS POLICY governance.city_access_policy 
    AS (city STRING) 
    RETURNS BOOLEAN ->
    CURRENT_ROLE() IN ('ACCOUNTADMIN', 'COMPLIANCE_OFFICER') OR
    (CURRENT_ROLE() = 'DATA_ANALYST' AND city = 'San Francisco');

-- Apply policy to members table
ALTER TABLE members ADD ROW ACCESS POLICY governance.city_access_policy ON (city);


In [ ]:
-- Test 1: COMPLIANCE_OFFICER (unmasked PII, all cities)
USE ROLE COMPLIANCE_OFFICER;
USE DATABASE PATELCO_DEMO; USE SCHEMA CREDIT_UNION_DATA; USE WAREHOUSE COMPUTE_WH;

SELECT 
    'COMPLIANCE_OFFICER_FULL_ACCESS' as test_case,
    member_id,
    first_name,    -- Should show real names
    last_name,     -- Should show real names
    city,          -- Should show all cities
    state,         -- All CA
    annual_income  -- Should show real income
FROM members 
ORDER BY city, member_id
LIMIT 10;


In [ ]:
-- Test 2: DATA_ANALYST (masked PII, San Francisco only)
USE ROLE DATA_ANALYST;
USE DATABASE PATELCO_DEMO; USE SCHEMA CREDIT_UNION_DATA; USE WAREHOUSE COMPUTE_WH;

SELECT 
    'DATA_ANALYST_RESTRICTED_ACCESS' as test_case,
    member_id,
    first_name,    -- Should be ****MASKED****
    last_name,     -- Should be ****MASKED****
    city,          -- Should only show San Francisco
    state,         -- All CA
    annual_income  -- Should be NULL
FROM members 
ORDER BY city, member_id
LIMIT 10;


In [ ]:
-- Test 3: Switch DATA_ANALYST to Oakland (masked PII, Oakland only)
USE ROLE accountadmin;
USE DATABASE PATELCO_DEMO; USE SCHEMA CREDIT_UNION_DATA; USE WAREHOUSE COMPUTE_WH;

-- Flip to Oakland
UPDATE governance.row_policy_map 
SET city_permission = 'Oakland' 
WHERE UPPER(role) = 'DATA_ANALYST';

-- Test as DATA_ANALYST (should see only Oakland, masked)
USE ROLE DATA_ANALYST;
SELECT 
    'DATA_ANALYST_OAKLAND_ONLY' as test_case,
    member_id,
    first_name,    -- Should be ****MASKED****
    last_name,     -- Should be ****MASKED****
    city,          -- Should only show Oakland
    state,         -- All CA
    annual_income  -- Should be NULL
FROM members 
ORDER BY city, member_id
LIMIT 10;

-- Revert to San Francisco
USE ROLE accountadmin;
UPDATE governance.row_policy_map 
SET city_permission = 'San Francisco' 
WHERE UPPER(role) = 'DATA_ANALYST';


In [ ]:
/*  5. Data Quality Monitoring with DMFs  */

USE ROLE accountadmin;
USE DATABASE PATELCO_DEMO; USE SCHEMA CREDIT_UNION_DATA; USE WAREHOUSE COMPUTE_WH;

-- Create schema for custom DMFs
CREATE SCHEMA IF NOT EXISTS data_quality;

-- Grant DMF execution privileges
GRANT EXECUTE DATA METRIC FUNCTION ON ACCOUNT TO ROLE compliance_officer;
GRANT CREATE DATA METRIC FUNCTION ON SCHEMA data_quality TO ROLE compliance_officer;
GRANT USAGE ON SCHEMA data_quality TO ROLE compliance_officer;

-- Custom DMF for email validation
CREATE OR REPLACE DATA METRIC FUNCTION data_quality.email_validation(
    input_data TABLE(email STRING)
) RETURNS NUMBER AS
$$
SELECT COUNT(*) 
FROM input_data 
WHERE email NOT LIKE '%@%.%' AND email IS NOT NULL
$$;

-- Custom DMF for negative income detection
CREATE OR REPLACE DATA METRIC FUNCTION data_quality.negative_income_count(
    input_data TABLE(annual_income NUMBER)
) RETURNS NUMBER AS
$$
SELECT COUNT(*) 
FROM input_data 
WHERE annual_income < 0
$$;

-- Custom DMF for income statistics
CREATE OR REPLACE DATA METRIC FUNCTION data_quality.income_stats(
    input_data TABLE(annual_income NUMBER)
) RETURNS OBJECT AS
$$
SELECT OBJECT_CONSTRUCT(
    'min', MIN(annual_income),
    'max', MAX(annual_income),
    'avg', ROUND(AVG(annual_income))
)
FROM input_data
$$;

-- Set DMF schedule on members table
ALTER TABLE members SET DATA_METRIC_SCHEDULE = 'TRIGGER_ON_CHANGES';

-- Apply system DMFs for completeness
ALTER TABLE members ADD DATA METRIC FUNCTION SNOWFLAKE.CORE.NULL_PERCENT
    ON (first_name, last_name, email, annual_income);

-- Apply system DMF for duplicates
ALTER TABLE members ADD DATA METRIC FUNCTION SNOWFLAKE.CORE.DUPLICATE_COUNT
    ON (member_id);

-- Apply custom DMFs for data validation
ALTER TABLE members ADD DATA METRIC FUNCTION data_quality.email_validation
    ON (email);
ALTER TABLE members ADD DATA METRIC FUNCTION data_quality.negative_income_count
    ON (annual_income);
ALTER TABLE members ADD DATA METRIC FUNCTION data_quality.income_stats
    ON (annual_income);

-- Switch to compliance officer to view metrics
USE ROLE compliance_officer;

-- View current metrics
WITH dmf_results AS (
  SELECT 
    -- System DMFs for completeness
    ROUND(100 - SNOWFLAKE.CORE.NULL_PERCENT(SELECT first_name FROM members), 1) as first_name_complete_pct,
    ROUND(100 - SNOWFLAKE.CORE.NULL_PERCENT(SELECT last_name FROM members), 1) as last_name_complete_pct,
    ROUND(100 - SNOWFLAKE.CORE.NULL_PERCENT(SELECT email FROM members), 1) as email_complete_pct,
    ROUND(100 - SNOWFLAKE.CORE.NULL_PERCENT(SELECT annual_income FROM members), 1) as income_complete_pct,
    
    -- System DMF for duplicates
    SNOWFLAKE.CORE.DUPLICATE_COUNT(SELECT member_id FROM members) as duplicate_count,
    
    -- Custom DMFs for validation
    data_quality.email_validation(SELECT email FROM members) as invalid_emails,
    data_quality.negative_income_count(SELECT annual_income FROM members) as negative_incomes,
    data_quality.income_stats(SELECT annual_income FROM members) as income_stats,
    
    -- Basic counts
    (SELECT COUNT(*) FROM members) as total_members,
    (SELECT COUNT(DISTINCT member_id) FROM members) as unique_members
)
SELECT
    '📊 MEMBER COUNT' as metric,
    total_members || ' total, ' || 
    unique_members || ' unique (' ||
    CASE WHEN duplicate_count = 0 THEN '✅' ELSE '❌' END || 
    ' duplicates: ' || duplicate_count || ')' as value
FROM dmf_results
UNION ALL
SELECT 
    '📋 COMPLETENESS',
    first_name_complete_pct || '% first names, ' ||
    last_name_complete_pct || '% last names, ' ||
    email_complete_pct || '% emails, ' ||
    income_complete_pct || '% incomes' 
FROM dmf_results
UNION ALL
SELECT
    '⚠️ DATA ISSUES',
    invalid_emails || ' invalid emails, ' ||
    negative_incomes || ' negative incomes'
FROM dmf_results
UNION ALL
SELECT
    '💰 INCOME RANGE',
    '$' || income_stats:min || ' to $' || income_stats:max || 
    ' (avg: $' || income_stats:avg || ')'
FROM dmf_results
ORDER BY 1;


### 6. Trust Center (Monitoring)
Open Monitoring → Trust Center in Snowsight to enable scanner packages (CIS Benchmarks, Threat Intelligence). This demo keeps Trust Center steps manual to stay minimal.

---
Run cells 1→6 in order. Then run the “Action demo” queries in sections 3 and 4 to show masking and RLS with `COMPLIANCE_OFFICER` vs `DATA_ANALYST`. Cortex services will automatically honor these policies.
